In [1]:
import pandas as pd
import numpy as np
import folium as fm
import re
import os
import simplejson as json
import urllib

In [8]:
os.path.exists('./Data/P3_GrantExport.csv')
data = pd.read_csv('./Data/P3_GrantExport.csv', delimiter=';', na_values=['.'], error_bad_lines=False)

We drop all unuseful columns

In [9]:
data.drop('Project Title', axis=1, inplace=True)
data.drop('Project Title English', axis=1, inplace=True)
data.drop('Responsible Applicant', axis=1, inplace=True)
data.drop('Funding Instrument', axis=1, inplace=True)
data.drop('Funding Instrument Hierarchy', axis=1, inplace=True)
data.drop('Institution', axis=1, inplace=True)
data.drop('Discipline Number', axis=1, inplace=True)
data.drop('Discipline Name', axis=1, inplace=True)
data.drop('Discipline Name Hierarchy', axis=1, inplace=True)
data.drop('Keywords', axis=1, inplace=True)

We drop project with missing starting date or ending date

In [360]:
len(data)

63969

In [10]:
data = data[pd.notnull(data['Start Date'])]
data = data[pd.notnull(data['End Date'])]
data.head()

,"﻿""Project Number""",University,Start Date,End Date,Approved Amount
0,1,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,4,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,5,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,6,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,7,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


In [362]:
len(data)

63968

We drop the "Nicht zuteilbar" in the university column

In [11]:
data = data[data.University != 'Nicht zuteilbar - NA']

In [364]:
len(data)

61373

In [12]:
data = data.dropna(axis=0, subset=['University'])

In [366]:
len(data)

48392

Converting the "Approved Amount" column to float type:

In [14]:
def convert_to_number(nb):
    try:
        return float(nb)
    except ValueError:
        return 0.0

data['Approved Amount'] = data['Approved Amount'].apply(convert_to_number)

In [15]:
a = [['a', 'a', '4.2'], ['b', '70.2', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df

,one,two,three
0,a,a,4.2
1,b,70.2,0.03
2,x,5,0


In [16]:
data2 = data.groupby('University').sum()
data2.drop('Project Number', axis=1, inplace=True)
data2.head()

ValueError: labels ['Project Number'] not contained in axis

In [17]:
universities = data['University'].unique()

university_names = universities.copy()

In [18]:
ids = {}
for record in university_names:
    wds = str(record).split()
    name = ' '.join(wds[:-2])
    if name:
        ids[name] = wds[-1]

In [372]:
cantons_id = ['ZH', 'BE','LU', 'UR', 'SZ', 'OW','NW', 'GL', 'ZG', 'FR',
                          'SO','BS', 'BL', 'SH', 'AR', 
                          'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD',
                          'VS', 'NE', 'GE','JU']

In [373]:
cantons_name = ['Zürich', 'Bern/Berne', 'Luzern', 'Uri','Schwyz', 'Obwalden', 
                'Nidwalden','Glarus', 'Zug', 'Fribourg', 'Solothurn','Basel-Stadt', 'Basel-Landschaft', 'Schaffhausen',
               'Appenzell Ausserrhoden','Appenzell Innerrhoden', 'St. Gallen', 'Graubünden/Grigioni', 'Aargau','Thurgau',
               'Ticino','Vaud', 'Valais/Wallis','Neuchâtel', 'Genève', 'Jura']

Spliting universities names and abbreviation and putting everything inside the dataframe

In [375]:
universities_pd = data2.copy()
university_names = universities_pd.index.values

name_list = []
abbrev_list = []
canton_list = []

for i, uni in enumerate(university_names):
    split_list = str(uni).split(' - ')
    if(len(split_list) == 1):
        split_list.append('')
    
    uni_name = split_list[0]
    uni_abrev = split_list[1]
    
    ## Infer the canton ID if we can find it in the University name
    uni_canton = np.nan
    if uni_abrev in cantons_id:
        uni_canton = uni_abrev
    else:
        found_match = False
        for canton_index, canton in enumerate(cantons_name):
            if canton in uni_name:
                if found_match:
                    print(uni_canton, ' - ', canton)
                    uni_canton = np.nan
                else:
                    found_match = True
                    uni_canton = cantons_id[canton_index]
        
    name_list.append(uni_name)
    abbrev_list.append(uni_abrev)
    canton_list.append(uni_canton)
    
    
    
universities_pd['university name'] = name_list
universities_pd['university abbrev'] = abbrev_list

# canton_list = []
# for canton_abrev in universities_pd['university abbrev']:
#     if canton_abrev not in cantons_id:
#         canton_abrev = ''
#     canton_list.append(canton_abrev)
    
# print(canton_list)

universities_pd['canton'] = canton_list
# universities_pd['canton'] = universities_pd['university abbrev'] if universities_pd['university abbrev'] in cantons_id
universities_pd

,Approved Amount,university name,university abbrev,canton
University,,,,
AO Research Institute - AORI,3.435621e+06,AO Research Institute,AORI,NaN
Allergie- und Asthmaforschung - SIAF,1.916996e+07,Allergie- und Asthmaforschung,SIAF,NaN
Berner Fachhochschule - BFH,3.102870e+07,Berner Fachhochschule,BFH,NaN
Biotechnologie Institut Thurgau - BITG,2.492535e+06,Biotechnologie Institut Thurgau,BITG,TG
Centre de rech. sur l'environnement alpin - CREALP,1.567678e+06,Centre de rech. sur l'environnement alpin,CREALP,NaN
EPF Lausanne - EPFL,1.175316e+09,EPF Lausanne,EPFL,NaN
ETH Zürich - ETHZ,1.635597e+09,ETH Zürich,ETHZ,ZH
Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,Eidg. Anstalt für Wasserversorgung,EAWAG,NaN
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",4.836039e+07,"Eidg. Forschungsanstalt für Wald,Schnee,Land",WSL,NaN


In [384]:
def look_for_missing_canton(canton):
    if canton is np.nan:
        return 'aaaaa'
    else:
        return canton
test = universities_pd.copy()
test['canton'].apply(look_for_missing_canton).head()

University
AO Research Institute - AORI                          aaaaa
Allergie- und Asthmaforschung - SIAF                  aaaaa
Berner Fachhochschule - BFH                           aaaaa
Biotechnologie Institut Thurgau - BITG                   TG
Centre de rech. sur l'environnement alpin - CREALP    aaaaa
Name: canton, dtype: object

In [19]:
import simplejson as json
DOMAIN = 'http://api.geonames.org/'#api.geonames.org/search?
USERNAME = 'shynkaru' #geonames username

def geonames_query(method, params):
    uri = DOMAIN + '%s?%s&username=%s' % (method, urllib.parse.urlencode(params), USERNAME)
    resource = urllib.request.urlopen(uri).readlines()
    return json.loads(resource[0])

In [20]:
def prepare_params(university, format_type):
    return { 'q' : university
            , 'country' : "ch"
            , 'lang': 'en'
            , 'featurecode':'univ'
            , 'type':format_type}

In [21]:
def try_query(university):
    if not university:
        return None
    
    params = prepare_params(university, 'json')
    response_json = geonames_query('search', params)
    
    if(not response_json['geonames']):
        return None
        #return try_query(' '.join(university.split()[:-1]))
    return response_json

In [22]:
def query_university(university):
    response_json = try_query(university)
    if(not response_json):
        return None
   
    out_json = response_json['geonames']
    total_results = int(response_json['totalResultsCount'])
    
    l = [( out_json[i]['name']
          , out_json[i].get('adminCode1', 'Missing: adminCode1')
          , out_json[i]['adminName1']
          , out_json[i]['lat']
          , out_json[i]['lng']) for i in range(len(out_json))]
        
    return l

In [23]:
def process_data():
    for university, canton in ids.items():
        print(university)
        possible_uni_data = query_university(university)
        if not possible_uni_data:
            print(None, '\n')
            continue
            
        for item in possible_uni_data:
            print(item)
        print('\n')

process_data()

Université de Fribourg
('Fribourg, Université', 'FR', 'Fribourg', '46.80683', '7.15317')


Pädagogische Hochschule Zürich
None 

Staatsunabh. Theologische Hochschule Basel
None 

Inst. de Hautes Etudes Internat. et du Dév
None 

Pädagogische Hochschule Thurgau
None 

Ente Ospedaliero Cantonale
None 

Haute école pédagogique fribourgeoise
None 

Eidg. Hochschulinstitut für Berufsbildung
None 

Facoltà di Teologia di Lugano
None 

Universität Basel
('University of Basel', 'BS', 'Basel-City', '47.55832', '7.58403')
('Universität', 'BS', 'Basel-City', '47.55707', '7.58405')


Interkant. Hochschule für Heilpädagogik ZH
None 

EPF Lausanne
None 

SUP della Svizzera italiana
None 

Pädagogische Hochschule Wallis
None 

Pädagogische Hochschule Nordwestschweiz
None 

Pädagogische Hochschule Luzern
None 

Fachhochschule Kalaidos
None 

Zürcher Fachhochschule (ohne PH)
None 

Inst. universit. romand de Santé au Travail
None 

Swiss Institute of Bioinformatics
None 

HES de Suisse occidentale
None

In [26]:
#Constants

URL = 'https://maps.googleapis.com/maps/api/place'
AUTOCOMPLETE_API = 'autocomplete'
DETAILS_API = 'details'

FORMAT = 'json'
KEY = ''#ask me the key

To get Geo Data by University Name we use:

request uri: 'https://maps.googleapis.com/maps/api/place/autocomplete'
format json

REQUEST PARAMETERS:

input= < university_name >
components = [ country < ISO 3166-1 Alpha-2 compatible country code >, ... ]
key = < API key>

E.g.:
https://maps.googleapis.com/maps/api/place/autocomplete/json?input=Universit%C3%A4t+Luzern&components=country:CH&types='administrative_area_level_1&key=

In [27]:
def google_maps_query(api, format_type, params, key):
    uri = URL + '/%s/%s?%s&key=%s' % (api, format_type, urllib.parse.urlencode(params), key)
    resource = urllib.request.urlopen(uri).read()
    return json.loads(resource)

In [28]:
def prepare_gm_uni_params(university):
    return {'input': university, 'components' : 'country:CH'}

def prepare_gm_placeid_params(place_id):
    return {'placeid': place_id, 'result_type':'administrative_area_level_1'}

In [29]:
def query_uni_gm(university, format_type, key):
    if not university:
        return None
    
    params = prepare_gm_uni_params(university)
    return google_maps_query(AUTOCOMPLETE_API, format_type, params, key)

def query_placeid_gm(place_id, format_type, key):
    if not place_id:
        return None
    
    params = prepare_gm_placeid_params(place_id)
    return google_maps_query(DETAILS_API, format_type, params, key)

In [30]:
def get_canton(university):
    uni_data = query_uni_gm(university, FORMAT, KEY)
    print(uni_data)
    if 'ZERO_RESULTS' == uni_data['status']:
        return None
       
    res_json = json.dumps(uni_data['predictions'][0], ensure_ascii=False)
    place_id = uni_data['predictions'][0]['place_id']
    place_id_data = query_placeid_gm(place_id, FORMAT, KEY)
    print(place_id_data)
    #'types':[ 'administrative_area_level_1', 'political']
    for node in place_id_data['result']['address_components']:
        types_node = node['types']
        if 'administrative_area_level_1' in types_node and 'political' in types_node:
            return (node['long_name'], node['short_name'])
    
    return None

In [31]:
def process_uni_data():
    i = 0
    for university, canton in ids.items():
        print(university)
        canton_data = get_canton(university)
        print(university, canton_data)
        if canton_data is not None:
            i+=1
    print(i)

process_uni_data()

Université de Fribourg
{'predictions': [], 'status': 'REQUEST_DENIED', 'error_message': 'This service requires an API key.'}


IndexError: list index out of range